### < Topic >

1. review
    
    1.1 1일차 내용에 대한 review (naver 영화 한줄평 이용)


2. topic
    
    2.1 정수 인코딩 (Integer encoding)
    
    2.2 원 핫 인코딩
    
    2.3 Byte Pair Encoding (BPE)
    
    2.4 데이터 분리 (Splitting  data)

 참고) 강사님 메일: wolee777@aicore.co.kr 

# Review

# 1. Naver 영화 한줄평 crawling

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321#'

In [7]:
resp = requests.get(url)
html = BeautifulSoup(resp.content, 'lxml')

In [8]:
score_result = html.find('div', {'class' : 'score_result'})
lis = score_result.findAll('li')

In [9]:
# li 태그가 여러 개 이므로 반복문으로 리스트에 담음
# strip(): 문자열 앞뒤 공백을 제거하고 내용만 가져오는 함수

comments = []
for li in lis:
    review_text = li.find('p').getText()
    comments.append(review_text.strip())
print(comments)

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


In [10]:
# 줄바꿈 기호를 넣어 보기 좋게 처리

for comment in comments:
    print(comment + '\n')

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.

충무로: 이거 어케하는거냐?

아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.

촬영감독의 영혼까지 갈아넣은 마스터피스

오스카 작품상 탔어도 할말 없었을것 같다.



### - Crawling 결과를 파일에 저장 / 읽기

In [11]:
# 문장 단위로 저장
# text 파일에 저장 시 한 줄씩 저장하는 것이 좋음

with open ('comment_1917.txt', 'w', encoding = 'utf-8') as f:
    for comment in comments:
        f.write( comment + '\n')

In [12]:
# readlines(): 실제 읽어들이는 내용 - 전체 내용을 list로 생성
# rstrip: 줄바꿈 기호 제거 - 나중에 처리할 때 문제될 수 있음

movie_comments = []

with open('comment_1917.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines:
        movie_comments.append(line.rstrip('\n'))

print(movie_comments)

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


### - 결과 리스트 내용을 하나의 문자열로 변경
리스트는 벡터 파일이기 때문에 하나씩 인덱스로 접근해야 함.

In [13]:
movie_comment = ''

for comment in movie_comments:
    movie_comment += comment

print(movie_comment)

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.충무로: 이거 어케하는거냐?아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.


# 2. Text preprocessing

## 1) Tokenize

In [14]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import re

In [15]:
okt = Okt()

### - 형태소 추출

In [16]:
morphemes = []

for morphs in okt.morphs(movie_comment):
    morphemes.append(morphs)

print(morphemes)

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


### - 형태소 추출 및 어간 추출

In [9]:
morphemes = []

for morphs in okt.morphs(movie_comment, stem = True):
    morphemes.append(morphs)

print(morphemes)

['이', '영화', '는', '미치다', '.', '넷플릭스', '가', '일상', '화', '되다', '시대', '에', '극장', '이', '존재', '하다', '이유', '를', '증명', '해주다', '.', '충무로', ':', '이', '거', '어케', '하다', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받다', '이유', '가', '고스', '란', '히', '녹다', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '하다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣다', '마스터피스', '오스카', '작품', '상', '타다', '하다', '없다', '같다', '.']


### - 형태소 추출 및 정규화

In [46]:
morphemes = []

for morphs in okt.morphs(movie_comment, norm = True):
    morphemes.append(morphs)

print(morphemes)

['이', '영화', '는', '미쳤다', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '충무로', '이', '거', '어떻게', '하는거냐', '아카데미', '에서', '촬영상', '음향효과', '상', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다']


In [34]:
# cleaning: 형태소 분석 이전, preprocessing 등 언제나 할 수 있음
# 꼭 cleaning 이후 형태소 분석을 할 필요는 없음
# noise data 제거는 항상 필요

### - 명사 분리

In [35]:
morphemes = []

for morphs in okt.nouns(movie_comment):
    morphemes.append(morphs)

print(morphemes)

['이', '영화', '넷플릭스', '일상', '시대', '극장', '존재', '이유', '증명', '충무로', '거', '어케', '아카데미', '촬영상', '음향효과', '효과', '이유', '고스', '란', '영화', '로', '관람', '는걸', '추천', '촬영감독', '영혼', '마스터피스', '오스카', '작품']


### - 어절 분리

In [36]:
morphemes = []

for morphs in okt.phrases(movie_comment):
    morphemes.append(morphs)

print(morphemes)

['이 영화', '넷플릭스', '일상화', '일상화된 시대', '극장', '존재', '존재해야하는 이유', '증명', '충무로', '충무로 이거', '충무로 이거 어케', '아카데미', '촬영상', '촬영상 음향효과상', '촬영상 음향효과상 시각효과상', '이유', '고스란', '영화', '영화 IMAX로', '영화 IMAX로 관람하는걸', '영화 IMAX로 관람하는걸 추천', '촬영감독', '촬영감독의 영혼', '마스터피스오스카', '마스터피스오스카 작품상', '시대', '이거', '어케', '음향효과상', '시각효과상', '고스', 'IMAX', '관람하', '는걸', '추천', '영혼', '마스터피스', '오스카', '작품상']


### - 품사 태깅

In [37]:
morphemes = []

for morphs in okt.pos(movie_comment, join =True):  # join 옵션을 안 주면 태그(/Noun)가 같이 안 나옴
    morphemes.append(morphs)

print(morphemes)

['이/Noun', '영화/Noun', '는/Josa', '미쳤다/Adjective', '넷플릭스/Noun', '가/Josa', '일상/Noun', '화/Suffix', '된/Verb', '시대/Noun', '에/Josa', '극장/Noun', '이/Josa', '존재/Noun', '해야하는/Verb', '이유/Noun', '를/Josa', '증명/Noun', '해준다/Verb', '충무로/Noun', '이/Determiner', '거/Noun', '어케/Noun', '하는거냐/Verb', '아카데미/Noun', '에서/Josa', '촬영상/Noun', '음향효과/Noun', '상/Suffix', '시/Modifier', '각/Modifier', '효과/Noun', '상/Suffix', '을/Josa', '받은/Verb', '이유/Noun', '가/Josa', '고스/Noun', '란/Noun', '히/Adverb', '녹아있는/Verb', '영화/Noun', 'IMAX/Alpha', '로/Noun', '관람/Noun', '하/Suffix', '는걸/Noun', '추천/Noun', '한다/Verb', '촬영감독/Noun', '의/Josa', '영혼/Noun', '까지/Josa', '갈아/Adverb', '넣은/Verb', '마스터피스/Noun', '오스카/Noun', '작품/Noun', '상/Suffix', '탔어도/Verb', '할말/Verb', '없었을것/Adjective', '같다/Adjective']


## 2) 불용어 처리 방법 1

### - 불용어 파일로 직접 지정

https://www.ranks.nl/stopwords/korean

- 위 사이트에서 불용어 복사해, visual studio, 메모장 등에 붙여넣고 파일 생성
- 마지막 줄 비워놓기

In [17]:
# with 문은 자동으로 close 수행 (파일 읽고, append할 때 유용)

stop_words = []

with open('stopwords.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.rstrip('\n'))
        
# print(stop_words)

In [18]:
# 아래 데이터에는 .과 ?가 포함되는 문제점 (noise data)

results = []

for word in okt.morphs(movie_comment):
    if word not in stop_words:
        results.append(word)

print(results)

['영화', '는', '미쳤다', '.', '넷플릭스', '일상', '화', '된', '시대', '극장', '존재', '해야하는', '이유', '증명', '해준다', '.', '충무로', ':', '거', '어케', '하는거냐', '?', '아카데미', '촬영상', ',', '음향효과', '상', ',', '시', '효과', '상', '받은', '이유', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '관람', '는걸', '추천', '한다', '.', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '.']


## 불용어 처리 방법 2

### - 정규식 이용 (cleaning)

In [19]:
r = '[-=+,#/\?:^$.@*\"~&%.!\'|\(\)\[\]\<\>`\']'

movie_comment = re.sub(r, '', movie_comment)
print(movie_comment)

이 영화는 미쳤다 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다충무로 이거 어케하는거냐아카데미에서 촬영상 음향효과상 시각효과상을 받은 이유가 고스란히 녹아있는 영화 IMAX로 관람하는걸 추천한다촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다


## 불용어 처리 방법 3

### - NLTK.word_tokenize() 사용

In [20]:
word_tokens = word_tokenize(movie_comment)
print(word_tokens)

['이', '영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것', '같다']


In [21]:
results = []
for word in word_tokens:
    if word not in stop_words:
        results.append(word)
print(results)

['영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것']


## 3) konlpy 실습

참조: http://konlpy.org/ko/latest/

In [39]:
from konlpy.corpus import kolaw
from konlpy.corpus import kobill

In [40]:
kolaw.open( 'constitution.txt' ).read()[ :30 ]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국'

In [41]:
kobill.open( '1809890.txt' ).read()[ :30 ]

'지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n'

### - 한나눔 (Hannanum) 형태소 분석기

In [42]:
from konlpy.tag import Hannanum

In [43]:
hannanum = Hannanum()

In [44]:
hannanum.nouns('나는 자연어 처리를 공부한다.')

['나', '자연어', '처리', '공부']

In [45]:
hannanum.pos('나는 자연어 처리를 공부한다.')

[('나', 'N'),
 ('는', 'J'),
 ('자연어', 'N'),
 ('처리', 'N'),
 ('를', 'J'),
 ('공부', 'N'),
 ('하', 'X'),
 ('ㄴ다', 'E'),
 ('.', 'S')]

# Topic: Encoding

### 컴퓨터의 인식을 수월히 하기 위해서는 인코딩 필요 (텍스트 -> 숫자)

# 1. 정수 인코딩 (Integer encoding)

- 텍스트를 정수로 변환하는 방법
- 텍스트의 빈도수를 확인해서 빈도수에 따른 index 부여 (decending)
- 단어를 빈도수 순으로 정렬한 단어 집합 (vocabulary)을 만들고, 빈도수가 높은 순서부터 차례로 낮은 숫자의 정수를 부여하는 방법

In [22]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [29]:
text="A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

## 1) dictionary 사용하기

### - 텍스트 전처리: 토큰화, 불용어 처리, cleaning

In [30]:
# 문장 토큰화

text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [37]:
# 정제와 단어 토큰화

vocab = {}  # dictionary 자료형: 단어-빈도수 저장
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i)  # 단어 토큰화
    result = []
    
    for word in sentence:
        word = word.lower()  # 소문자 변환 함수
        if word not in stop_words:  # 불용어 제거
            if len(word) > 2:   # 단어 길이가 2 이하인 경우, 단어 제거
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0   # 단어 없으면 0
                vocab[word] += 1     # 단어 있으면 +1
    sentences.append(result)

print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


### - 빈도수 계산

In [34]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [38]:
# 해당 단어의 빈도수 확인 가능

print(vocab['good'])
print(vocab['word'])

1
2


### - 빈도수가 높은 순서대로 정렬

In [40]:
# sorted() 함수
# dictionary는 key를 대상으로 할 것인지, value를 대상으로 할 것인지 정해야 함
# items(): 항목 하나씩 꺼냄
# sorted() 함수의 key라는 인수에 정렬식을 넣음
# lambda 인수(x):return 값(x[1]) -> 1번째 데이터를 꺼냄

vocab_sorted = sorted(vocab.items(), key = lambda x: x [1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


### - 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여 

In [43]:
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:   # dictionary는 key, value로 이루어짐
    if frequency > 1:  # 빈도수가 일정 이하인 단어는 제외
        i += 1
        word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


### - 상위 빈도 단어만 저장 (5위)

In [44]:
# list comprehension 사용: 특별한 제어문 없이 데이터 구조 손쉽게 표현
# w: key(단어) / c: value(빈도수)

vocab_size = 5
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1]

for w in words_frequency:
    del word_to_index[w]  # 해당 단어에 대한 인덱스 정보 삭제

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### - Out-Of-Vocabulary (OOV, 단어 집합에 없는 단어)
train, test set에는 있으나 단어 집합에 존재하지 않는 단어들(저빈도 단어들)의 처리 필요

In [45]:
# 'OOV' 추가

word_to_index['OOV'] = len(word_to_index) + 1

In [46]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


### - Mapping: word_to_index(단어: 빈도수)로 sentences(전처리 후 데이터)의 모든 단어 처리

In [48]:
encoded = []

for s in sentences:
    temp = []
    for w in s:  # sentences(토믄화된 모든 단어) 중 word_to_index의 key값 검색
        try:
            temp.append(word_to_index[w])
        except KeyError:   # word_to_index의 key에 포함되지 않는 단어들
            temp.append(word_to_index['OOV'])   # OOV에 담음
    encoded.append(temp)

print(encoded)   # 결과 값은 각 문장들을 치환

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


## 2) Counter 사용하기

In [50]:
from collections import Counter

In [51]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


### - 단어 집합(vocabulary) 만들기: sentences에서 문장의 경계인 '[ ]'를 제거하고 단어 전부를 담은 하나의 리스트 생성

In [53]:
# sum(iterable): 입력받은 리스트나 튜플의 모든 요소의 합을 돌려주는 함수

words = sum(sentences, [])  # '[]'는 제외하고 새로운 리스트를 만듦
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


### - Counter()을 이용하여 중복을 제거하고 빈도수 기록

In [55]:
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


### - most_common(): 상위 빈도 단어만 저장 (5위)

In [57]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) ; vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

### - 상위 빈도 단어일수록 낮은 정수 인덱스 부여

In [59]:
word_to_index = {}
i = 0

for (word, frequency) in vocab:   # dictionary는 key, value로 이루어짐
    i += 1
    word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### - 이후, OOV 처리와 mapping까지 하면 완료

## (3) NLTK의 FreqDist 사용하기

In [60]:
from nltk import FreqDist
import numpy as np

### - 빈도수 저장

In [61]:
# np.hstack(): 문장 구분 제거
# FreqDist 형식으로 객체를 담음

vocab = FreqDist(np.hstack(sentences)) ; vocab

FreqDist({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, ...})

In [62]:
print(vocab['barber'])

8


### - most_common(): 상위 빈도 단어만 저장 (5위)

In [65]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) ; vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

### - 상위 빈도 단어일수록 낮은 정수 인덱스 부여

In [66]:
# list comprehension 형식
# key: value
# enumerate: 데이터를 차례대로 꺼냄

word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)                                      

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### - 이후, OOV 처리와 mapping까지 하면 완료

## 참고) enumerate 함수

- Iterator: 반복자, 반복 기능을 포함한 객체
           Sequence 자료형: list, tuple, dictionary, string

In [85]:
# enunemrate 사용

test = ['a', 'b', 'c', 'd', 'e']

for index, value in enumerate(test):
    print("value : {}, index: {}".format(value, index))

print('--------------------')
    
# enumerate 사용 X

index = 0
for value in test:
    print("value : {}, index: {}".format(value, index))
    index += 1

value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4
--------------------
value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4


# + Keras가 제공하는 Text preprocessing
### - tokenize / 빈도수에 따른 index 부여 / 정수 인덱싱

In [86]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [87]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [89]:
# 객체 생성

tokenizer = Tokenizer()

# fir_on_texts(): 빈도수 기준으로 단어 집합을 생성하는 함수(method)

tokenizer.fit_on_texts(sentences)

In [90]:
# 각 단어에 부여된 인덱스 확인

print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [92]:
# 각 단어의 개수 확인

print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [94]:
# 정수 인코딩

print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


### - 빈도수에 따른 상위 5개 추출 / 정수 인덱싱

In [100]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [111]:
vocab_size = 5

# 삭제할 대상
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1]

# 삭제
for w in words_frequency:
    del tokenizer.word_index[w]
    del tokenizer.word_counts[w]

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


### - 더 간단한 방법) 빈도수에 따른 상위 5개 추출 / 정수 인덱싱

In [112]:
tokenizer = Tokenizer(num_words=vocab_size+1)  # (0~4) 상위 5개만 사용
tokenizer.fit_on_texts(sentences)

In [113]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [114]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


# 2. One-hot Encoding
- 단어 집합의 크기를 벡터(1차원 배열)의 차원으로 표현
- 0과 1로 표현
- "정수 인코딩"을 한 후, "원핫 인코딩"

단어 집합( 5 : 빈도수가 높은 상위 단어 수): ['barber', 'person', 'huge', 'kept', 'secret']

단어 집합의 단어에 대한 인덱스 부여: [ 1   /    2    /    3    /    4    /    5  ]
   
정수 인코딩:                [10000  /  01000  /   00100  /  00010  /  00001]

### - 1) tokenize

In [117]:
from konlpy.tag import Okt

In [118]:
okt = Okt()

In [121]:
token = okt.morphs('나는 자연어 처리를 배운다')
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


### - 2) 정수 인코딩

In [124]:
# 여기서는 빈도수 의미 없음

word2index = {}

for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)

print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


### - 3) One-hot Encoding

In [126]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index))
    index = word2index[word]
    one_hot_vector[index]=1

    return one_hot_vector

In [127]:
one_hot_encoding("자연어", word2index)

[0, 0, 1, 0, 0, 0]

# + Keras가 제공하는 One-hot-encoding

In [128]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [137]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

### - 1) 정수 인코딩 (단어 집합 생성)

In [138]:
t = Tokenizer()
t.fit_on_texts([text])
print(t.word_index)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [139]:
sub_text = '점심 먹으러 갈래 메뉴는 햄버거 최고야'

encoded = t.texts_to_sequences([sub_text])[0]
print(encoded)

[2, 5, 1, 6, 3, 7]


### - 2) One-hot-Encoding

In [140]:
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [142]:
# 앞줄의 0은 keras에서 0을 먼저 부여하는 특징 때문
# one-hot vector size: 7개이므로 8(7+1)열
# '점심 먹으러 갈래 메뉴는 햄버거 최고야' -> 6개의 token: 6행

## One-hot encoding의 한계

1. 단어 집합에 인덱스가 부여된 단어 수만큼 one-hot vector size가 결정 (메모리 낭비)

2. 유사 단어에 대한 처리가 불가능

3. one-hot encoding 한계점을 극복하기 위해 사용하는 방법: 
 
    - 카운트 기반 벡터 방식
    - 예측 기반 벡터 방식

# 3. BPE(Byte Pair Encoding)

- OOV (Out-Of-Vocabulary) / UNK(Unknown Token) 문제 해결
- 압축 알고리즘: 중복되는 부분을 줄임
- 중복되는 단어를 모아 압축

### 실습 과제: 
1. 네이버 영화 한줄평 크롤링 결과를 이용해서 2, 3일차 내용 적용
2. 네이버 영화에서 영화별로 줄거리를 크롤링하고, 각각을 파일로 저장, 그 결과를 이용해서 2, 3일차 내용 적용

(tokenize, normalization, stop_words, integer encoding, one-hot-encoding)